In [18]:
# Imports (External)
import numpy as np
import pandas as pd
import datetime
from datetime import datetime

from monthdelta import monthdelta
import xlrd
import xlsxwriter
from collections import OrderedDict
import pickle

import sys
sys.path.append('../')  

# Visualization/plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt

# Machine learning imports
import pywt
from pywt import wavedec, waverec
from scipy import signal
from statsmodels.robust import mad

import sklearn
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import optimizers
from keras import regularizers

# Internal Imports
from wsae_lstm.utils import pickle_load

In [19]:
dict_dataframes_index=pickle_load(path_filename="../data/interim/cdii_tvt_split.pickle")
dict_dataframes_index.keys()

dict_keys(['csi300 index data', 'nifty 50 index data', 'hangseng index data', 'nikkei 225 index data', 's&p500 index data', 'djia index data'])

In [20]:
# Train
dict_dataframes_index['csi300 index data'][1][1].head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08


In [21]:
# Validate
dict_dataframes_index['csi300 index data'][1][2].head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2010-07-01,2557.92,2574.10,2524.57,2526.07,328576.46,84.55,2.2492,-68.405561,-199.061153,49.525,2742.562654,2713.585325,2691.0766,-232.424,2626.8448,-223.952,-8.143641,-0.063088,-9.298846e+07
2010-07-02,2523.04,2542.12,2462.20,2534.11,449141.00,84.37,2.1942,-74.447502,-160.377588,79.920,2729.510192,2696.491961,2674.8705,-223.393,2586.4084,-224.761,-8.146862,-0.088669,-8.628590e+07


In [22]:
# Test
dict_dataframes_index['csi300 index data'][1][3].head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2010-10-08,2965.65,3055.84,2963.24,3044.23,999947.20,77.20,1.7325,14.071181,187.594016,120.262,2912.040962,2913.539251,2898.0211,194.405,2928.1110,111.679,3.808250,0.043045,1.079573e+08
2010-10-11,3069.19,3154.20,3069.19,3132.90,1587349.04,77.47,1.6683,29.910782,243.000652,109.971,2920.543577,2934.430275,2919.9912,275.419,2973.6842,170.576,5.758191,0.087812,2.313821e+08


In [23]:
len(dict_dataframes_index['csi300 index data'][1][1].columns)
# features will be 19 (20-1=19), with date column removed 
    # with rangeindex used = data is still indexed, but rangeindex not counted as column

19

In [24]:
data = dict_dataframes_index['csi300 index data'][1][1]

# Stacked Autoencoders
## Notes from source article on Stacked Autoencoder design:

"The single-layer autoencoder maps the input daily variables into the first hidden vector." 

"In this paper, [the activate function] is set to be a sigmoid function as in Chen et al. [19]"

"As a result, the gradient descent algorithm is applied to complete parameter optimization as suggested in Yin et al. [62]"

"After training the first single-layer autoencoder, the reconstruction layer of the first single layer autoencoder is removed, and the hidden layer is reserved as the input layer of the second single-layer autoencoder." (Bao et al., 2017, p. 7-8)


"SAEs is the main part of the
model and is used to learn the deep features of financial time series in an unsupervised manner. Specifically, it is a neural network consisting of multiple single layer autoencoders in
which the output feature of each layer is wired to the inputs of the successive layer. The unsupervised training of SAEs is done one AE at a time by minimizing the error between the output
data and the input data. As a result, the SAEs model can successfully learn invariant and
abstract features [19]." (Bao et al., 2017, p.2)

## Implementation notes & summary:
**Stacked Autoencoder with 5 Layers that consists of 4-single layer autoencoders**

- Input variables for each dataset range from 18-25 features
- Depth of 5, hidden layer size of 10
- Gradient descent for parameter optimization
- Unsupervised training

## Hypotheses from meta-referenced articles for model & implementation details: 
- 100 Epochs for training (Chen et al., 2014)

In [25]:
input_dim = data.shape[1]
hidden_dim = 10  

# Layer 1 - Input Layer, SAE_1
input_data = Input(shape=(input_dim,))
encoded = Dense(hidden_dim, activation='sigmoid')(input_data)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
# Layer 2 - SAE_2
encoded = Dense(hidden_dim, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
# Layer 3 - SAE_3
encoded = Dense(hidden_dim, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
# Layer 4 - SAE_4
encoded = Dense(hidden_dim, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(encoded)
# Layer 5 - SAE_5
encoded = Dense(hidden_dim, activation='sigmoid')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='sgd',loss='mse')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 19)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                200       
_________________________________________________________________
dense_12 (Dense)             (None, 19)                209       
_________________________________________________________________
dense_13 (Dense)             (None, 10)                200       
_________________________________________________________________
dense_14 (Dense)             (None, 19)                209       
_________________________________________________________________
dense_15 (Dense)             (None, 10)                200       
_________________________________________________________________
dense_16 (Dense)             (None, 19)                209       
__________

In [26]:
autoencoder.fit(data, data, epochs=10,verbose=2)

Epoch 1/10
 - 1s - loss: 5543926704618213.0000
Epoch 2/10
 - 0s - loss: 5543926762806676.0000
Epoch 3/10
 - 0s - loss: 5543926691443467.0000
Epoch 4/10
 - 0s - loss: 5543926596201030.0000
Epoch 5/10
 - 0s - loss: 5543926706814004.0000
Epoch 6/10
 - 0s - loss: 5543926665093974.0000
Epoch 7/10
 - 0s - loss: 5543926648625542.0000
Epoch 8/10
 - 0s - loss: 5543926562166269.0000
Epoch 9/10
 - 0s - loss: 5543926688149781.0000
Epoch 10/10
 - 0s - loss: 5543926636548690.0000


In [27]:
autoencoded_data = autoencoder.predict(data)

In [28]:
pd.DataFrame(autoencoded_data).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.999653,0.999658,0.999648,0.999652,1.0,0.986332,0.505469,0.126277,0.804441,0.986074,0.999654,0.999653,0.999653,0.654762,0.999653,0.22467,0.770405,0.389829,1.0
1,0.999653,0.999658,0.999648,0.999652,1.0,0.986332,0.505469,0.126277,0.804441,0.986074,0.999654,0.999653,0.999653,0.654762,0.999653,0.22467,0.770405,0.389829,1.0
2,0.999653,0.999658,0.999648,0.999652,1.0,0.986332,0.505469,0.126277,0.804441,0.986074,0.999654,0.999653,0.999653,0.654762,0.999653,0.22467,0.770405,0.389829,1.0
3,0.999653,0.999658,0.999648,0.999652,1.0,0.986332,0.505469,0.126277,0.804441,0.986074,0.999654,0.999653,0.999653,0.654762,0.999653,0.22467,0.770405,0.389829,1.0
4,0.999653,0.999658,0.999648,0.999652,1.0,0.986332,0.505469,0.126277,0.804441,0.986074,0.999654,0.999653,0.999653,0.654762,0.999653,0.22467,0.770405,0.389829,1.0


In [29]:
len(pd.DataFrame(autoencoded_data).columns)

19